In [2]:
!git clone https://github.com/facebookresearch/segment-anything-2.git
%cd /kaggle/working/segment-anything-2
%pip install -e .
%cd /kaggle/working/segment-anything-2/checkpoints/
!bash /kaggle/working/segment-anything-2/checkpoints/download_ckpts.sh
%cd /kaggle/working/segment-anything-2/

Cloning into 'segment-anything-2'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Receiving objects: 100% (1070/1070), 134.70 MiB | 41.04 MiB/s, done.
Resolving deltas: 100% (375/375), done.
/kaggle/working/segment-anything-2
Obtaining file:///kaggle/working/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 5.3 MB/s eta 0:00:00
  Building editable for SAM-2 (pyproject.toml) ... done
  Created wheel for SAM-2: filename=SAM_2-1.0-0.editable-cp310-cp310-linux_x86_64.whl size=13818 sha256=f1b4a5a474dd7ec86087326b0911654cc01c4c53bb36813bdcf80c8233349496
  Stored in dire

In [1]:
!wget https://storage.yandexcloud.net/ds-ods/files/files/2ad601fd/test.zip
!wget https://storage.yandexcloud.net/ds-ods/files/submissions/21dd71fc-63e7-4419-8cc6-ec6af1214dee/62ab2c57/NTO_SUB0%202.zip
!pip install supervision

--2025-01-22 18:01:04--  https://storage.yandexcloud.net/ds-ods/files/files/2ad601fd/test.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1444862118 (1.3G) [application/x-zip-compressed]
Saving to: ‘test.zip’

test.zip            100%[===================>]   1.34G  20.3MB/s    in 70s     

2025-01-22 18:02:15 (19.6 MB/s) - ‘test.zip’ saved [1444862118/1444862118]

--2025-01-22 18:02:15--  https://storage.yandexcloud.net/ds-ods/files/submissions/21dd71fc-63e7-4419-8cc6-ec6af1214dee/62ab2c57/NTO_SUB0%202.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81104172 (77M) [application/zip]
Sav

In [5]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image

In [12]:
!unzip "/kaggle/working/NTO_SUB0 2.zip"

Archive:  /kaggle/working/NTO_SUB0 2.zip
   creating: NTO_SUB0.zip/
  inflating: __MACOSX/._NTO_SUB0.zip  
  inflating: NTO_SUB0.zip/run.py     
  inflating: __MACOSX/NTO_SUB0.zip/._run.py  
  inflating: NTO_SUB0.zip/.DS_Store  
  inflating: __MACOSX/NTO_SUB0.zip/._.DS_Store  
  inflating: NTO_SUB0.zip/best.pt    
  inflating: __MACOSX/NTO_SUB0.zip/._best.pt  
  inflating: NTO_SUB0.zip/metadata.json  
  inflating: __MACOSX/NTO_SUB0.zip/._metadata.json  


In [6]:
# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

using device: cuda


In [32]:
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

sam2_checkpoint = "./checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

sam2_model = build_sam2(model_cfg, sam2_checkpoint, device=device)

predictor = SAM2ImagePredictor(sam2_model)

In [39]:
!pip install ultralytics supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 4.0 MB/s eta 0:00:00a 0:00:01


In [49]:
from  ultralytics import YOLO
import supervision as sv
model = YOLO('NTO_SUB0.zip/best.pt')

def predict_yolo_image(image):
    return sv.Detections.from_ultralytics(model(image, augment=True,conf=0.5)[0])

In [75]:
def predict_sam(image,box):
    predictor.set_image(image)
    masks, scores, _ = predictor.predict(
        point_coords=None,
        point_labels=None,
        box=box,
        multimask_output=False,
    )
    return masks, scores

In [76]:
np.random.seed(3)

def show_mask(mask, ax, random_color=False, borders = True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask = mask.astype(np.uint8)
    mask_image =  mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    if borders:
        import cv2
        contours, _ = cv2.findContours(mask,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        # Try to smooth contours
        contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
        mask_image = cv2.drawContours(mask_image, contours, -1, (1, 1, 1, 0.5), thickness=2) 
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2))    

def show_masks(image, masks, scores, point_coords=None, box_coords=None, input_labels=None, borders=True):
    for i, (mask, score) in enumerate(zip(masks, scores)):
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        show_mask(mask, plt.gca(), borders=borders)
        if point_coords is not None:
            assert input_labels is not None
            show_points(point_coords, input_labels, plt.gca())
        if box_coords is not None:
            # boxes
            show_box(box_coords, plt.gca())
        if len(scores) > 1:
            plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
        plt.axis('off')
        plt.show()

In [ ]:
from tqdm.auto import tqdm
import supervision as sv
import pycocotools
import json
import pycocotools.mask
import numpy as np
from ultralytics import YOLO
import albumentations as A
import os

test_folder = "test"
output_folder = "output_yolo"  # Папка для сохранения результатов
os.makedirs(output_folder, exist_ok=True)
predictions = []

for file_name in tqdm(os.listdir(test_folder)):
    image_path = os.path.join(test_folder, file_name)
    image = Image.open(image_path)
    predicts = predict_yolo_image(image)

    if predicts.xyxy is None:
        continue
    pred_classes = predicts.class_id.tolist()  # Классы объектов
    pred_boxes = predicts.xyxy.tolist()#instances.pred_boxes.tensor.tolist()  # Координаты bbox
    pred_scores = predicts.confidence.tolist()  # Скоринг предсказаний
        
    for i in range(len(pred_boxes)):
        mask , scores = predict_sam(image,pred_boxes[i])
        rle = pycocotools.mask.encode(np.asfortranarray(mask.astype(np.uint8)[0]))
        predictions.append({
            "image_name": file_name,
            "category_id": 0,  # ID категории
            "bbox": pred_boxes[i],  # Координаты bounding box
            "score": scores[0],  # Оценка уверенности
            "segmentation": {
                "size" : rle["size"],
                "counts" : str(rle["counts"])
            }  # Сегментация в формате RLE
        })


# Сохранение предсказаний в JSON
predictions_file = os.path.join(output_folder, "submission.json")
with open(predictions_file, "w") as f:
    json.dump(predictions, f, indent=4)

print(f"Обработка завершена. Результаты сохранены в папке: {output_folder}")

  0%|          | 0/9074 [00:00<?, ?it/s]


0: 416x768 1 Оленевые, 131.7ms
Speed: 3.1ms preprocess, 131.7ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 768)

0: 416x768 1 Медведь, 119.6ms
Speed: 2.7ms preprocess, 119.6ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 768)

0: 416x768 1 Полорогие, 119.5ms
Speed: 2.5ms preprocess, 119.5ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 768)

0: 416x768 1 Медведь, 120.1ms
Speed: 2.7ms preprocess, 120.1ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 768)

0: 416x768 1 Оленевые, 1 Пантеры, 119.6ms
Speed: 2.5ms preprocess, 119.6ms inference, 1.9ms postprocess per image at shape (1, 3, 416, 768)

0: 416x768 1 Заяц, 1 Собачие, 119.5ms
Speed: 2.5ms preprocess, 119.5ms inference, 1.5ms postprocess per image at shape (1, 3, 416, 768)

0: 416x768 1 Кошки, 119.4ms
Speed: 2.4ms preprocess, 119.4ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 768)

0: 416x768 2 Оленевыеs, 119.1ms
Speed: 2.6ms preprocess, 119.1ms inference,